In [7]:
import json
with open('/home/garimachahar/learnTensorflow/sarcasm.json', 'r') as f:
    datastore= json.load(f)

sentences=[]
labels=[]
urls=[]
for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])
    urls.append(item['article_link'])

In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [9]:
vocab_size=10000    #assuming max unique words 10000. if there are more than 10000 words they will be ignored
embedding_dim=16    #basically vector (word representation) will be in 16 dimensions
max_length=32      #max width of padded matrix (i.e. truncated width of each sentence)
trunc_type='post'
padding_type= 'post'
oov_tok= "<OOV>"   #token given to words in sequence which are not encountered before while fitting
training_size=20000

In [10]:
#splitting for train and test data. Total= 27000. train=20000
training_sentences= sentences[0:training_size]
testing_sentences= sentences[training_size:]
training_labels= labels[0:training_size]
testing_labels= labels[training_size:]

In [16]:
tokenizer= Tokenizer(num_words=vocab_size, oov_token= '<OOV>')
tokenizer.fit_on_texts(training_sentences)
word_index= tokenizer.word_index
#getting encoded code for sentences
sequences= tokenizer.texts_to_sequences(training_sentences)

#input size feeded to NN should be uniform. For that PADDING is done!
#list of sentences have been padded out into sentences

#for padding after the sentences
padded= pad_sequences(sequences, maxlen= max_length, padding=padding_type, truncating= trunc_type)

testing_sequences= tokenizer.texts_to_sequences(testing_sentences)
testing_padded= pad_sequences(testing_sequences, maxlen= max_length, padding=padding_type, truncating= trunc_type)

In [12]:
#GlobalAveragePooling1D is similar to Flatten() but takes less time beacuse its output is smaller in size
import tensorflow as tf
model= tf.keras.Sequential([ tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length), tf.keras.layers.GlobalAveragePooling1D(), tf.keras.layers.Dense(24, activation= 'relu'), tf.keras.layers.Dense(1, activation= 'sigmoid')])

model.compile(loss='binary_crossentropy', optimizer= 'adam', metrics= ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 32, 16)            160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 24)                408       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 160,433
Trainable params: 160,433
Non-trainable params: 0
_________________________________________________________________


In [17]:
num_epochs=30
history= model.fit(padded, training_labels, epochs= num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

ValueError: Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, (<class 'list'> containing values of types {"<class 'int'>"})

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabels(string)
    plt.legend([string, 'val_'+string])
    plt.show()
    
plot_graphs()

In [ ]:
#to understand LOSS is indirectly proportional to confidence in
#predicting
""" with above parametrs LOSS for validation kept increasing.. 
so change parameter.... so just trying different combination of
parameter combination to have 90+ accuracy %"""
vocab_size=1000  #was 10000
max_length=16  #was 32
embedding_dim= 32  #was 16